In [19]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
n_inputs = 4
n_hidden = 20
n_classes = 3

num_samples = 150

In [21]:
def logits_function(p, X):
    # Roll-back the weights and biases
    W1 = p[0:80].reshape((n_inputs,n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden,n_classes))
    b2 = p[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [22]:
# Forward propagation
def forward_prop(params, X,y):
    logits = logits_function(params, X)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [23]:
def Get_PSO(f):
    # Initialize swarm
    options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

    # Call instance of PSO
    dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
    optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=1000)
    return pos

In [24]:
def predict(pos, X):
    logits = logits_function(pos, X)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [25]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

labelencoder = LabelEncoder()

def Start_Iris():
    data = pd.read_csv('databases/iris.data', names = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])
    target = labelencoder.fit_transform(data['class'].values)
    data = data.drop('class', axis = 1).values

    def f(x):
        n_particles = x.shape[0]
        j = [forward_prop(x[i], data, target) for i in range(n_particles)]
        return np.array(j)

    pos = Get_PSO(f)
    scores = (predict(pos, data) == target)

    print("\n Iris \n")
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())


In [26]:
Start_Iris()

2022-03-07 19:47:19,004 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.00953
2022-03-07 19:47:54,203 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.009534524901048214, best pos: [ 1.61878380e+00 -1.01484291e+00  5.29846847e-01 -3.50938255e-01
  1.58073248e+00 -2.73379126e+00 -2.43602188e-01  2.55859315e-01
  1.94896117e-01 -4.67694273e-02  1.79796632e+00  4.94371143e+00
  6.20332720e-01  6.41653520e-01  1.54553581e-01  2.34701326e+00
  1.07295015e+00  4.88103177e-01  6.40680890e-01  7.12845771e-02
  3.75353796e-01 -3.38938558e-02 -4.74836644e-01 -5.75055111e-01
 -3.30262737e+00  6.11839508e-01  1.62900904e-01  5.66227096e-01
 -3.20582513e-01 -8.01336920e-01 -3.31170041e-01  7.38761983e-01
 -6.03059507e-01  1.92764793e+00  3.98837082e-01  1.25446464e+01
  1.64453380e+00  4.76702208e-01  5.56667564e-01 -5.34050902e-01
  5.44219521e-01 -1.1


 Iris 

Mean:  1.0
Standard Deviation:  0.0
Min:  True
Max:  True
